In [1]:
import os
import numpy as np
import pandas as pd
from mpl_toolkits import mplot3d
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [2]:
ROOT_FOLDER = os.path.join(os.getcwd(), "plot_data")

# Robustness

In [3]:
FILENAME = "robustness_350_class_8_small.csv"
FOLDER_PATH = os.path.join(ROOT_FOLDER, "robustness_data")
CSV_PATH = os.path.join(FOLDER_PATH, FILENAME)

df = pd.read_csv(CSV_PATH)
total_shift = np.add(np.abs(df["Shift_X"]), np.abs(df["Shift_Y"]))

In [4]:
max_row = df.iloc[df['mIoU'].idxmax()]
hline_str = f"mIoU: {max_row['mIoU']}, Shift: {max_row['Shift_X']},{max_row['Shift_Y']}"
print(f"Best record: {max_row}")

min_iou = df['mIoU'].min()
max_iou = df['mIoU'].max()

Best record: Angle      -0.050
Shift_X   -60.000
Shift_Y    80.000
mIoU        0.844
Name: 1070, dtype: float64


In [5]:
fig = px.scatter_3d(df, x='Angle', y=total_shift, z='mIoU',
              color='mIoU', width=1200, height=800, labels={"y":"Total Shift", "Angle": "Angle (rad)"}, range_color=[min_iou, max_iou])

fig.update_layout(title="Augmentation Robustness - Class 8")
fig.update_coloraxes(cmin=min_iou, cmax=max_iou)
fig.show()

In [6]:
fig = px.scatter(df, x="Angle", y="mIoU", color=total_shift,
                 width=1200, height=600, labels={"color":"Total Shift", "Angle": "Angle (rad)"})
fig.update_xaxes(tickvals=df["Angle"])
fig.update_layout(title="Augmentation Robustness - All classes")
fig.show()

# Threshold Plots

In [ ]:
FILENAME = "th_slice_var_500.csv"
FOLDER_PATH = os.path.join(ROOT_FOLDER, "threshold_data")
CSV_PATH = os.path.join(FOLDER_PATH, FILENAME)

df = pd.read_csv(CSV_PATH)

In [ ]:
max_row = df.iloc[df['IoU'].idxmax()]
hline_str = f"IoU: {np.round(max_row['IoU'], 3)}, Th_Values: {max_row['Th_Value']}"
print(f"Best record: {max_row}")

In [ ]:
fig = px.line(df, x="Th_Value", y="IoU", width=1200, height=600, markers=True)
fig.update_xaxes(tickvals=df["Th_Value"])
fig.add_trace(go.Scatter(x=[max_row['Th_Value']], y=[max_row['IoU']], mode = 'markers',
                         marker_symbol = 'star',
                         marker_size = 15,
                         showlegend=False))
fig.update_layout(title="Best threshold value for Slice Var mode")
fig.show()

# Sweep plot

In [ ]:
FILENAME = "slice_var_no_norm_no_l1.csv"
FOLDER_PATH = os.path.join(ROOT_FOLDER, "sweep_data")
CSV_PATH = os.path.join(FOLDER_PATH, FILENAME)

df = pd.read_csv(CSV_PATH)

In [ ]:
max_row = df.iloc[df['aug_iou_single'].idxmax()]
print(max_row)

In [ ]:
fig = px.scatter(df, x="lambda_L2", y="lambda_tv", color="aug_iou_single",
                 width=1200, height=600)
fig.show()

In [ ]:
sum = df["lambda_L2"] + df["lambda_tv"]
norm_lambda_L2 = df["lambda_L2"] / sum
norm_lambda_tv = df["lambda_tv"] / sum

fig = px.scatter(df, x=norm_lambda_L2, y=norm_lambda_tv, color="avg_aug_SR_iou",
                 width=1200, height=600, labels={"x":"Lambda L2", "y":"Lambda TV"})
fig.show()


In [ ]:
FILENAME = "slice_var_no_norm_l1.csv"
FOLDER_PATH = os.path.join(ROOT_FOLDER, "sweep_data")
CSV_PATH = os.path.join(FOLDER_PATH, FILENAME)

df = pd.read_csv(CSV_PATH)

In [ ]:
max_row = df.iloc[df['aug_iou_single'].idxmax()]
print(max_row)

In [ ]:
fig = px.scatter_3d(df, x='lambda_L1', y='lambda_L2', z='lambda_tv',
              color='aug_iou_single', width=1200, height=800)
fig.show()

In [ ]:
fig = px.scatter(df, x="lambda_L2", y="lambda_tv", color="aug_iou_single",
                 width=1200, height=600)
fig.show()

# Dropout Test

In [ ]:
FILENAME = "100_rep_500_samples_max.csv"
FOLDER_PATH = os.path.join(ROOT_FOLDER, "dropout_data")
CSV_PATH = os.path.join(FOLDER_PATH, FILENAME)

df = pd.read_csv(CSV_PATH)

In [ ]:
max_row = df.iloc[df['dIoU'].idxmax()]
print(max_row)

In [ ]:
fig = px.scatter(df, x="D_value", y="dIoU", width=1200, height=600)
fig.add_hline(y=0.829)
fig.show()

# Final Evaluations

In [3]:
FILENAME = "argmax_validation_final.csv"
CSV_PATH = os.path.join(ROOT_FOLDER, FILENAME)

df = pd.read_csv(CSV_PATH)

In [4]:
# df = df[::-1]
# index = np.arange(1, 21)
name = df["Name"]
aug_single = df["aug_iou_single"]
standard_single = df["standard_iou_single"]
mean_iou = df["mean_iou"]
max_iou = df["max_iou"]

fig = go.Figure()
fig.add_trace(go.Bar(
    x=name,
    y=aug_single,
    name="Aug. SR",
    marker_color='indianred'
))
fig.add_trace(go.Bar(
    x=name,
    y=standard_single,
    name="Standard",
    marker_color='lightsalmon'
))

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', xaxis = {'type' : 'category'}, xaxis_title="Class ID", yaxis_title="IoU")
fig.show()

In [ ]:
aug_single_avg = aug_single.mean()
std_single_avg = standard_single.mean()

In [ ]:
diff = aug_single - standard_single
avg = np.mean(diff)

print(avg * 100)

In [ ]:
diff = np.around(aug_single, 2) - np.around(standard_single, 2)
avg = np.mean(diff)

print(avg * 100)

In [ ]:
aug_single_perc = aug_single * 100
standard_single_perc = standard_single * 100

diff = aug_single_perc - standard_single_perc
avg = np.mean(diff)

print(avg)

In [ ]:
diff = np.around(aug_single_perc, 2) - np.around(standard_single_perc, 2)
avg = np.mean(diff)

print(avg)

In [7]:
diff_mean = mean_iou - standard_single
print(np.mean(diff_mean) * 100)

-0.01752415432613258


In [8]:
diff_max = max_iou - standard_single
print(np.mean(diff_max) * 100)

-5.204053502277282
